In [537]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

conn = psycopg2.connect(os.getenv('DATABASE_URL'))

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("""
SELECT *, date_trunc('hour', timestamp) as hour FROM trades
where sol_volume > 0.1
order by timestamp desc
--limit 50000
""")

# Retrieve query results
records = cur.fetchall()

# Convert to pandas DataFrame
df = pd.DataFrame(records, columns=[desc[0] for desc in cur.description])

# Close communication with the database
cur.close()
conn.close()

original_df = df.copy()

In [548]:
original_df.mint.describe()

count                                            81579
unique                                            2715
top       DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7YaB1pPB263
freq                                              6709
Name: mint, dtype: object

In [554]:
# get keys with at least 10 sol_volume and 10 trades
per_mint_data = original_df[['mint', 'sol_volume', 'trades']].groupby('mint').sum()
big_mints = per_mint_data[(per_mint_data['sol_volume'] >= 10) & (per_mint_data['trades'] >= 10)].index
df = original_df[original_df['mint'].isin(big_mints)]

df.mint.describe()

count                                            75227
unique                                             688
top       DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7YaB1pPB263
freq                                              6709
Name: mint, dtype: object

In [555]:
# all uniq pairs of mint and hour in separate columns in keys dataframe
keys = df[['mint', 'hour']].drop_duplicates()
keys

,mint,hour
0,DhRQWsiP53gVXnG8KQwq63Uo1FvouRpVNL8536tsVD5H,2024-01-02 19:00:00
3,65nTNuJGHme4PQvKQyJykKp1bJAkK4A8Q66sd2yBWugf,2024-01-02 19:00:00
4,562W2R6aFggFGZsEsReJo89GEAJm7XhQZWQxHuqkKHBD,2024-01-02 19:00:00
8,3FQqnC42Z8ykMxeRnYVj6sGoPmrhVURCG8ot7EPWLYjX,2024-01-02 19:00:00
10,E2WYCGJJtWBodVLy1NKcN8ve4UAtsJJBU2mdErbXxP8h,2024-01-02 19:00:00
...,...,...
81574,5SD4mnz5kNhnAmVQKKX911XBY6RJcqCD4pkH699bP5U3,2023-11-01 00:00:00
81575,DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7YaB1pPB263,2023-11-01 00:00:00
81576,D7rcV8SPxbv94s3kJETkrfMrWqHFs6qrmtbiu6saaany,2023-11-01 00:00:00
81577,AZsHEMXd36Bj1EMNXhowJajpUXzrKcK57wW4ZGXVa7yR,2023-11-01 00:00:00


In [556]:
# print min and max dates
print(f"min date: {df['timestamp'].min()}, max date: {df['timestamp'].max()}")

min date: 2023-11-01 00:14:03, max date: 2024-01-02 19:08:48


In [557]:
pre_prices = df[['mint', 'hour', 'trades', 'token_volume', 'sol_volume']]

# dataframe prices show price of each mint at each hour
# it's calculated as sum of token_volume divided by sum of sol_volume for each mint at each hour
prices = pre_prices.groupby(['mint', 'hour']).sum().reset_index()
prices['price'] = prices['sol_volume'] / prices['token_volume']
prices

,mint,hour,trades,token_volume,sol_volume,price
0,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-16 10:00:00,1,1.295834e+11,0.599024,4.622694e-12
1,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-16 12:00:00,1,1.663637e+11,0.500006,3.005502e-12
2,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-16 14:00:00,1,1.344338e+11,0.549258,4.085712e-12
3,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-20 04:00:00,2,1.746382e+12,2.140559,1.225711e-12
4,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-20 05:00:00,1,3.011002e+11,0.210124,6.978527e-13
...,...,...,...,...,...,...
37942,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-11 07:00:00,1,1.507630e+03,0.162542,1.078130e-04
37943,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-25 11:00:00,1,4.178073e+03,0.301122,7.207201e-05
37944,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-27 12:00:00,1,1.623808e+03,0.130625,8.044334e-05
37945,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-29 12:00:00,1,3.000000e+04,2.378402,7.928006e-05


In [558]:
from datetime import timedelta

prediction_types = [
    {'suffix': '_2h', 'delta': timedelta(hours=2)},
    {'suffix': '_6h', 'delta': timedelta(hours=6)},
    {'suffix': '_24h', 'delta': timedelta(hours=24)},
    {'suffix': '_7d', 'delta': timedelta(days=7)},
]

for prediction in prediction_types:
    # list of -1s with length of prices
    prices['price' + prediction['suffix']] = [-1.] * len(prices)

prices_map = {}
for index, row in prices.iterrows():
    mint = row['mint']
    hour = row['hour']
    prices_map[(mint, hour)] = row['price']

sol_volume_map = {}
for index, row in prices.iterrows():
    mint = row['mint']
    hour = row['hour']
    sol_volume_map[(mint, hour)] = row['sol_volume']

min_hour = prices['hour'].min()
max_hour = prices['hour'].max()

# fill gaps in prices_map
for mint in keys['mint'].unique():
    # from min to max hour
    hour = min_hour
    last_price = 0.
    while hour <= max_hour:
        if (mint, hour) not in prices_map:
            prices_map[(mint, hour)] = last_price
        else:
            last_price = prices_map[(mint, hour)]
        hour += timedelta(hours=1)

# fill gaps in sol_volume_map
for mint in keys['mint'].unique():
    # from min to max hour
    hour = min_hour
    cum_sol_volume = 0.
    while hour <= max_hour:
        cum_sol_volume += sol_volume_map.get((mint, hour), 0.)
        sol_volume_map[(mint, hour)] = cum_sol_volume        
        hour += timedelta(hours=1)

max_hour = prices['hour'].max()
# iterate over mint and hour pairs
for index, row in prices.iterrows():
    # get mint and hour from row
    mint = row['mint']
    hour = row['hour']
    for p in prediction_types:
        prediction_hour = hour + p['delta']
        # if there's no data for prediction_hour, skip
        if prediction_hour > max_hour:
            continue
        future_price = prices_map.get((mint, prediction_hour), -1)
        assert(future_price != -1)
        prices.loc[index, 'price' + p['suffix']] = future_price            


In [569]:
prices

,mint,hour,trades,token_volume,sol_volume,price,price_2h,price_6h,price_24h,price_7d
0,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-16 10:00:00,1,1.295834e+11,0.599024,4.622694e-12,3.005502e-12,4.085712e-12,4.085712e-12,4.571769e-12
1,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-16 12:00:00,1,1.663637e+11,0.500006,3.005502e-12,4.085712e-12,4.085712e-12,4.085712e-12,3.556662e-12
2,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-16 14:00:00,1,1.344338e+11,0.549258,4.085712e-12,4.085712e-12,4.085712e-12,4.085712e-12,3.556662e-12
3,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-20 04:00:00,2,1.746382e+12,2.140559,1.225711e-12,4.953954e-12,2.735042e-12,2.555918e-12,5.370183e-12
4,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-20 05:00:00,1,3.011002e+11,0.210124,6.978527e-13,3.400636e-12,2.735042e-12,2.555918e-12,5.370183e-12
...,...,...,...,...,...,...,...,...,...,...
37942,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-11 07:00:00,1,1.507630e+03,0.162542,1.078130e-04,1.078130e-04,1.078130e-04,1.078130e-04,1.078130e-04
37943,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-25 11:00:00,1,4.178073e+03,0.301122,7.207201e-05,7.207201e-05,7.207201e-05,7.207201e-05,7.928006e-05
37944,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-27 12:00:00,1,1.623808e+03,0.130625,8.044334e-05,8.044334e-05,8.044334e-05,8.044334e-05,-1.000000e+00
37945,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-29 12:00:00,1,3.000000e+04,2.378402,7.928006e-05,7.928006e-05,7.928006e-05,7.928006e-05,-1.000000e+00


In [572]:
from dataclasses import dataclass
from datetime import datetime, timedelta

FEATURES_PERIOD = 14

features = keys.copy()

def get_base_price(inp_mint, inp_hour):
    ans = prices_map.get((inp_mint, inp_hour), -1)
    assert(ans != -1)
    return ans


def get_price_points(inp_mint, last_hour, interval, num_points):
    ans = []
    for i in range(num_points):
        hour = last_hour - i * interval
        if hour < min_hour:
            break
        ans.append(get_base_price(inp_mint, hour))
    # reverse list
    ans = ans[::-1]
    return ans

def get_volume_points(inp_mint, last_hour, interval, num_points):
    ans = []
    for i in range(num_points):
        hour = last_hour - i * interval
        prev_hour = hour - interval
        if hour < min_hour:
            break
        ans.append(sol_volume_map.get((inp_mint, hour), 0.) - sol_volume_map.get((inp_mint, prev_hour), 0.))
    # reverse list
    ans = ans[::-1]
    return ans

def sma_price(inp_mint, last_hour, interval, num_points):
    price_points = get_price_points(inp_mint, last_hour, interval, num_points)
    return sum(price_points) / len(price_points) / get_base_price(inp_mint, last_hour)

def ema_price(inp_mint, last_hour, interval, num_points):
    price_points = get_price_points(inp_mint, last_hour, interval, num_points)
    k = 2 / (num_points + 1)
    ans = price_points[0]
    for i in range(1, len(price_points)):
        ans = k * price_points[i] + (1 - k) * ans
    return ans / get_base_price(inp_mint, last_hour)

def relative_strength_index(inp_mint, last_hour, interval, num_points):
    price_points = get_price_points(inp_mint, last_hour, interval, num_points)
    gains = []
    losses = []
    for i in range(1, len(price_points)):
        diff = price_points[i] - price_points[i - 1]
        if diff > 0:
            gains.append(diff)
        elif diff < 0:
            losses.append(-diff)
    if len(gains) == 0:
        return 0.
    if len(losses) == 0:
        return 100.
    avg_gain = sum(gains) / len(gains)
    avg_loss = sum(losses) / len(losses)
    return 100 - 100 / (1 + avg_gain / avg_loss)

def sma_volume(inp_mint, last_hour, interval, num_points):
    volume_points = get_volume_points(inp_mint, last_hour, interval, num_points)
    return sum(volume_points) / len(volume_points)

def ema_volume(inp_mint, last_hour, interval, num_points):
    volume_points = get_volume_points(inp_mint, last_hour, interval, num_points)
    k = 2 / (num_points + 1)
    ans = volume_points[0]
    for i in range(1, len(volume_points)):
        ans = k * volume_points[i] + (1 - k) * ans
    return ans

@dataclass
class FeatureAggr:
    name: str
    delta: timedelta
    num_points: int

aggregations = [
    FeatureAggr('1h', timedelta(hours=1), 14),
    FeatureAggr('4h', timedelta(hours=4), 14),
    FeatureAggr('1d', timedelta(hours=24), 14),
]

for a in aggregations:
    features['sma_' + a.name] = [-1.] * len(features)
    features['ema_' + a.name] = [-1.] * len(features)
    features['rsi_' + a.name] = [-1.] * len(features)
    for index, row in features.iterrows():
        mint = row['mint']
        hour = row['hour']
        features.loc[index, 'sma_' + a.name] = sma_price(mint, hour, a.delta, a.num_points)
        features.loc[index, 'ema_' + a.name] = ema_price(mint, hour, a.delta, a.num_points)
        features.loc[index, 'rsi_' + a.name] = relative_strength_index(mint, hour, a.delta, a.num_points)

#calculate volume features
for a in aggregations:
    features['sma_vol_' + a.name] = [-1.] * len(features)
    features['ema_vol_' + a.name] = [-1.] * len(features)
    for index, row in features.iterrows():
        mint = row['mint']
        hour = row['hour']
        features.loc[index, 'sma_vol_' + a.name] = sma_volume(mint, hour, a.delta, a.num_points)
        features.loc[index, 'ema_vol_' + a.name] = ema_volume(mint, hour, a.delta, a.num_points)

In [573]:
features.sort_values(by=['mint', 'hour'])

,mint,hour,sma_1h,ema_1h,rsi_1h,sma_4h,ema_4h,rsi_4h,sma_1d,ema_1d,rsi_1d,sma_vol_1h,ema_vol_1h,sma_vol_4h,ema_vol_4h,sma_vol_1d,ema_vol_1d
49204,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-16 10:00:00,0.071429,0.133333,100.000000,0.071429,0.133333,100.000000,0.071429,0.133333,100.000000,0.042787,0.079870,0.042787,0.079870,0.042787,0.079870
49079,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-16 12:00:00,0.291154,0.465102,74.082988,0.071429,0.133333,100.000000,0.071429,0.133333,100.000000,0.078502,0.126659,0.078502,0.146537,0.078502,0.146537
48954,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-16 14:00:00,0.338149,0.475319,63.810228,0.152245,0.264076,89.592712,0.071429,0.133333,100.000000,0.117735,0.168369,0.117735,0.209122,0.117735,0.219772
40497,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-20 04:00:00,3.166673,3.022229,0.000000,3.166673,3.022229,0.000000,0.785716,1.141665,58.823507,0.152897,0.285408,0.152897,0.285408,0.270632,0.428471
40484,12AjTG16m3oWzjVy2kP2C1bUEzZv8PayQDM9csRR1obM,2023-12-20 05:00:00,5.215192,4.733819,0.000000,5.507927,5.207398,0.000000,1.326005,1.904370,54.668804,0.167906,0.275370,0.167906,0.313424,0.285641,0.456488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56650,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-11 07:00:00,1.055331,1.052771,14.736419,1.181180,1.144888,5.800634,1.369964,1.319035,0.000000,0.032203,0.037961,0.504694,0.378897,0.511837,0.831017
22950,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-25 11:00:00,1.460485,1.429786,0.000000,1.460485,1.429786,0.000000,1.460485,1.429786,0.000000,0.021509,0.040150,0.021509,0.040150,0.021509,0.040150
16931,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-27 12:00:00,0.903368,0.909810,100.000000,0.935104,0.978954,18.977298,1.252461,1.199033,18.977298,0.009330,0.017417,0.030839,0.024626,0.030839,0.047573
11371,zebeczgi5fSEtbpfQKVZKCJ3WgYXxjkMUkNNx7fLKAF,2023-12-29 12:00:00,1.013625,1.012717,0.000000,1.006083,0.996284,87.799386,1.220472,1.164406,31.208969,0.169886,0.317120,0.179216,0.320248,0.200725,0.352853


In [665]:
PRICE_THRESHOLD = 4

for p in prediction_types:
    prices['class' + p['suffix']] = ['na'] * len(prices)
    
for index, row in prices.iterrows():
    for p in prediction_types:
        price = row['price']
        future_price = row['price' + p['suffix']]
        if future_price == -1:
            continue
        if future_price > PRICE_THRESHOLD * price:
            prices.loc[index, 'class' + p['suffix']] = 'up'
        elif future_price < price / PRICE_THRESHOLD:
            prices.loc[index, 'class' + p['suffix']] = 'down'
        else:
            prices.loc[index, 'class' + p['suffix']] = 'flat'

#sort prices by hour then mint
prices = prices.sort_values(by=['hour', 'mint'])

prices

,mint,hour,trades,token_volume,sol_volume,price,price_2h,price_6h,price_24h,price_7d,class_2h,class_6h,class_24h,class_7d
2045,3J5QaP1zJN9yXE7jr5XJa3Lq2TyGHSHu2wssK7N1Aw4p,2023-11-01 00:00:00,1,1.930000e+08,2.617965,1.356458e-08,1.356458e-08,1.356458e-08,9.007497e-09,4.647018e-09,flat,flat,flat,flat
4990,5SD4mnz5kNhnAmVQKKX911XBY6RJcqCD4pkH699bP5U3,2023-11-01 00:00:00,1,4.375231e+07,3.222574,7.365495e-08,9.118888e-08,6.138228e-08,1.789658e-08,1.789658e-08,flat,flat,down,down
9876,7kbnvuGBxxj8AG9qp8Scn56muWGaRaFqxg1FsRp3PaFT,2023-11-01 00:00:00,1,3.859803e+03,100.000006,2.590806e-02,2.590806e-02,2.590806e-02,2.362193e-02,2.423463e-02,flat,flat,flat,flat
12048,94XduSfSnyas7jAEFSJSXiCi1xQ4mENWcii1aCvjVuqu,2023-11-01 00:00:00,1,1.000000e+05,0.166931,1.669306e-06,3.096264e-06,3.096264e-06,3.877707e-06,4.437695e-06,flat,flat,flat,flat
12858,9WMwGcY6TcbSfy9XPpQymY3qNEsvEaYL3wivdwPG2fpp,2023-11-01 00:00:00,1,3.500819e+02,1.000006,2.856491e-03,2.856491e-03,2.856491e-03,2.856491e-03,2.188399e-03,flat,flat,flat,flat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8124,7AmejD1Jdox7YRLpeML7EdZ3JFGW78XBSM8xP6MEn6T2,2024-01-02 19:00:00,1,2.283524e+03,0.262717,1.150490e-04,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,na,na,na,na
12731,9L1hpsP3cqVeZjHcp9fXLWCTYeofeP52j9eEBQUFzbR8,2024-01-02 19:00:00,1,1.913543e+07,2.180041,1.139270e-07,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,na,na,na,na
19996,DhRQWsiP53gVXnG8KQwq63Uo1FvouRpVNL8536tsVD5H,2024-01-02 19:00:00,1,4.089509e+05,1.475644,3.608366e-06,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,na,na,na,na
20637,E2WYCGJJtWBodVLy1NKcN8ve4UAtsJJBU2mdErbXxP8h,2024-01-02 19:00:00,1,1.223751e+06,0.236705,1.934261e-07,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,na,na,na,na


In [666]:
td_names = ['class' + p['suffix'] for p in prediction_types]
classes = pd.concat([prices[name].value_counts() for name in td_names], axis=1)
classes.columns = td_names
classes

,class_2h,class_6h,class_24h,class_7d
flat,37501,36916,34313,22618
up,198,337,721,1612
down,175,406,1707,5366
na,73,288,1206,8351


In [667]:
from sklearn.model_selection import TimeSeriesSplit

prediction_type = prediction_types[2]
print(f"Prediction type: {prediction_type['suffix']}")

pool_df = prices.copy()
pool_df = pool_df[['mint', 'hour', 'trades', 'token_volume', 'sol_volume', 'price', 'class' + prediction_type['suffix']]]

len_before = len(pool_df)
# remove from pool rows with class 'na'
pool_df = pool_df[pool_df['class' + prediction_type['suffix']] != 'na']
len_after = len(pool_df)
print(f'Removed {len_before - len_after} of {len_before} rows with class "na"')

Prediction type: _24h
Removed 1206 of 37947 rows with class "na"


In [668]:
pool_df

,mint,hour,trades,token_volume,sol_volume,price,class_24h
2045,3J5QaP1zJN9yXE7jr5XJa3Lq2TyGHSHu2wssK7N1Aw4p,2023-11-01 00:00:00,1,1.930000e+08,2.617965,1.356458e-08,flat
4990,5SD4mnz5kNhnAmVQKKX911XBY6RJcqCD4pkH699bP5U3,2023-11-01 00:00:00,1,4.375231e+07,3.222574,7.365495e-08,down
9876,7kbnvuGBxxj8AG9qp8Scn56muWGaRaFqxg1FsRp3PaFT,2023-11-01 00:00:00,1,3.859803e+03,100.000006,2.590806e-02,flat
12048,94XduSfSnyas7jAEFSJSXiCi1xQ4mENWcii1aCvjVuqu,2023-11-01 00:00:00,1,1.000000e+05,0.166931,1.669306e-06,flat
12858,9WMwGcY6TcbSfy9XPpQymY3qNEsvEaYL3wivdwPG2fpp,2023-11-01 00:00:00,1,3.500819e+02,1.000006,2.856491e-03,flat
...,...,...,...,...,...,...,...
35067,iotEVVZLEywoTn1QdwNPddxPWszn3zFhEot3MfL9fns,2024-01-01 19:00:00,1,1.742559e+05,3.139126,1.801446e-05,flat
35488,jtojtomepa8beP8AuQc6eXt5FriJwfFMwQx2v2f9mCL,2024-01-01 19:00:00,1,5.724078e+00,0.103688,1.811443e-02,flat
35578,kiraZUmSnzgfVfhrdvNj6hxHFaPFTTUk8ioY98cbh6G,2024-01-01 19:00:00,1,1.100000e+03,0.538952,4.899561e-04,flat
36383,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,2024-01-01 19:00:00,4,2.258449e+00,2.609253,1.155330e+00,flat


In [669]:
print(pool_df.shape)
pool_df = pool_df.join(features.set_index(['mint', 'hour']), on=['mint', 'hour'])
print(pool_df.shape)

(36741, 7)
(36741, 22)


In [670]:
pool_df

,mint,hour,trades,token_volume,sol_volume,price,class_24h,sma_1h,ema_1h,rsi_1h,...,rsi_4h,sma_1d,ema_1d,rsi_1d,sma_vol_1h,ema_vol_1h,sma_vol_4h,ema_vol_4h,sma_vol_1d,ema_vol_1d
2045,3J5QaP1zJN9yXE7jr5XJa3Lq2TyGHSHu2wssK7N1Aw4p,2023-11-01 00:00:00,1,1.930000e+08,2.617965,1.356458e-08,flat,1.000000,1.000000,0.000000,...,0.000000,1.000000e+00,1.000000e+00,0.000000,2.617965,2.617965,2.617965,2.617965,2.617965,2.617965
4990,5SD4mnz5kNhnAmVQKKX911XBY6RJcqCD4pkH699bP5U3,2023-11-01 00:00:00,1,4.375231e+07,3.222574,7.365495e-08,down,1.000000,1.000000,0.000000,...,0.000000,1.000000e+00,1.000000e+00,0.000000,3.222574,3.222574,3.222574,3.222574,3.222574,3.222574
9876,7kbnvuGBxxj8AG9qp8Scn56muWGaRaFqxg1FsRp3PaFT,2023-11-01 00:00:00,1,3.859803e+03,100.000006,2.590806e-02,flat,1.000000,1.000000,0.000000,...,0.000000,1.000000e+00,1.000000e+00,0.000000,100.000006,100.000006,100.000006,100.000006,100.000006,100.000006
12048,94XduSfSnyas7jAEFSJSXiCi1xQ4mENWcii1aCvjVuqu,2023-11-01 00:00:00,1,1.000000e+05,0.166931,1.669306e-06,flat,1.000000,1.000000,0.000000,...,0.000000,1.000000e+00,1.000000e+00,0.000000,0.166931,0.166931,0.166931,0.166931,0.166931,0.166931
12858,9WMwGcY6TcbSfy9XPpQymY3qNEsvEaYL3wivdwPG2fpp,2023-11-01 00:00:00,1,3.500819e+02,1.000006,2.856491e-03,flat,1.000000,1.000000,0.000000,...,0.000000,1.000000e+00,1.000000e+00,0.000000,1.000006,1.000006,1.000006,1.000006,1.000006,1.000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35067,iotEVVZLEywoTn1QdwNPddxPWszn3zFhEot3MfL9fns,2024-01-01 19:00:00,1,1.742559e+05,3.139126,1.801446e-05,flat,1.064907,1.055609,0.000000,...,33.667369,1.203141e+00,1.248823e+00,19.328394,0.339518,0.596113,1.952009,2.156837,26.451202,34.628548
35488,jtojtomepa8beP8AuQc6eXt5FriJwfFMwQx2v2f9mCL,2024-01-01 19:00:00,1,5.724078e+00,0.103688,1.811443e-02,flat,0.980618,0.982468,82.341511,...,23.308232,1.347536e+00,1.315768e+00,27.749206,2.561340,2.576904,11.220160,11.878039,186.715113,149.920035
35578,kiraZUmSnzgfVfhrdvNj6hxHFaPFTTUk8ioY98cbh6G,2024-01-01 19:00:00,1,1.100000e+03,0.538952,4.899561e-04,flat,0.944439,0.967168,83.018777,...,89.754962,3.807236e-01,4.369517e-01,45.431530,0.078859,0.099530,0.157441,0.183846,0.480774,0.547720
36383,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,2024-01-01 19:00:00,4,2.258449e+00,2.609253,1.155330e+00,flat,1.001630,1.001233,62.331952,...,76.923077,6.312357e+06,1.021199e+07,46.153846,13.933845,16.066291,58.111424,58.413627,911.853747,815.276083


In [671]:
list(features.columns[2:])

['sma_1h',
 'ema_1h',
 'rsi_1h',
 'sma_4h',
 'ema_4h',
 'rsi_4h',
 'sma_1d',
 'ema_1d',
 'rsi_1d',
 'sma_vol_1h',
 'ema_vol_1h',
 'sma_vol_4h',
 'ema_vol_4h',
 'sma_vol_1d',
 'ema_vol_1d']

In [672]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Convert 'mint' to a categorical feature
pool_df['mint'] = pool_df['mint'].astype('category')

# Define features and target
# feature_names = ['mint', 'hour', 'trades', 'token_volume', 'sol_volume', 'price'] + list(features.columns[2:])
# cat_feature_names = ['mint']
# feature_names = ['trades', 'token_volume', 'sol_volume', 'price'] + list(features.columns[2:])
feature_names = list(features.columns[2:])
cat_feature_names = []
target = 'class' + prediction_type['suffix']

# Initialize CatBoost Classifier
model = CatBoostClassifier(loss_function='MultiClass', verbose=False)

# TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=2)

# To store predictions and actuals
actuals = []
raw_predictions = []
up_probabilities = []
details_df = pd.DataFrame() 
train_len = 0
up_train_len = 0


for train_index, test_index in tscv.split(pool_df):
    train_data, test_data = pool_df.iloc[train_index], pool_df.iloc[test_index]

    # mints that have at least 10 sol_volume in train_data
    big_mints = test_data[test_data['sma_vol_1d'] >= 5]['mint'].unique()
    print(f"big_mints: {len(big_mints)}")
    train_data = train_data[train_data['mint'].isin(big_mints)]
    test_data = test_data[test_data['mint'].isin(big_mints)]


    max_train_hour = train_data['hour'].max()
    len_before = len(test_data)
    # change those lines together please
    removed_data = test_data[test_data['hour'] <= max_train_hour + prediction_type['delta']]
    test_data = test_data[test_data['hour'] > max_train_hour + prediction_type['delta']]
    len_after = len(test_data)
    if len_after == 0:
        break
    print(f'Removed {len_before - len_after} of {len_before} rows from test data')
    print(f'since {min(removed_data["hour"])} to {max(removed_data["hour"])}')

    train_len += len(train_data)
    up_train_len += len(train_data[train_data[target] == 'up'])

    # Create Pool
    train_pool = Pool(data=train_data[feature_names], label=train_data[target], cat_features=cat_feature_names)
    test_pool = Pool(data=test_data[feature_names], label=test_data[target], cat_features=cat_feature_names)

    # Train model
    model.fit(train_pool)

    # Predict
    classes = model.classes_
    probs = model.predict_proba(test_pool)
    classes_to_index = {c: i for i, c in enumerate(classes)}
    
    for probs_row in probs:
        raw_predictions.append(classes[probs_row.argmax()])
        up_probabilities.append(probs_row[classes_to_index['up']])

    actuals.extend(test_data[target].values)
    if len(details_df) == 0:
        details_df = test_data
    else:
        details_df = pd.concat([details_df, test_data])
    


big_mints: 191
Removed 515 of 9923 rows from test data
since 2023-12-09 21:00:00 to 2023-12-10 21:00:00
big_mints: 244
Removed 1179 of 9714 rows from test data
since 2023-12-22 19:00:00 to 2023-12-23 19:00:00


In [679]:
desired_up_amount = int(up_train_len * 1. / train_len * len(actuals) * 0.1)
print(f"desired_up_amount: {desired_up_amount} of {actuals.count('up')}")

up_threshold = sorted(up_probabilities, reverse=True)[desired_up_amount]
print(f"up_threshold: {up_threshold}")

predictions = []
for raw_pred, up_prob in zip(raw_predictions, up_probabilities):
    if up_prob < up_threshold:
        predictions.append(raw_pred)
    else:
        predictions.append('up')

# Confusion Matrix
conf_matrix = confusion_matrix(actuals, predictions, labels=['up', 'flat', 'down'])
conf_matrix_df = pd.DataFrame(conf_matrix, index=['up', 'flat', 'down'], columns=['up', 'flat', 'down'])

# Last iteration's predictions and actuals
comparison_df = pd.DataFrame({'Actual': actuals, 'Predicted': predictions})
# comparison_df = comparison_df.reset_index(drop=True)

# add details to results_df
results_df = pd.concat([comparison_df.reset_index(drop=True), details_df.reset_index(drop=True)], axis=1)
print(f"pred len {len(predictions)}, actuals len {len(actuals)}, "
      f"comparison_df len {len(comparison_df)}, details len {len(details_df)}, results_df len {len(results_df)}")

#join prices to results_df
results_df = results_df.join(prices[['mint', 'hour', 'price'+prediction_type['suffix']]].set_index(['mint', 'hour']), on=['mint', 'hour'])
results_df['upside'] = results_df['price'+prediction_type['suffix']] / results_df['price']
upsides = results_df[(results_df['Predicted']=='up')&(results_df['upside']<1000)]['upside']


# Output
print("Confusion Matrix:\n", conf_matrix_df)
print("ROI1: %.2f%% of 100%%\n"
      "ROI2: %.2f%% of 100%%\n"
      "Benchmark: %.2f%% of 100%%\n"
      "Used: %.2f%% (%d of %d trades)\n"
      %(
        (100 * (PRICE_THRESHOLD * 1. * conf_matrix_df.loc['up', 'up'].sum() + conf_matrix_df.loc['flat', 'up'].sum()) / conf_matrix_df.loc[:, 'up'].sum()),
        (100 * (PRICE_THRESHOLD * 1. * conf_matrix_df.loc['up', 'up'].sum()) / conf_matrix_df.loc[:, 'up'].sum()),
        (100 * (PRICE_THRESHOLD * 1. * conf_matrix_df.loc['up', :].sum()) / conf_matrix_df.loc[:, :].sum().sum()),
        (100 * (1. * conf_matrix_df.loc[:, 'up'].sum()) / conf_matrix_df.loc[:, :].sum().sum()),
        conf_matrix_df.loc[:, 'up'].sum(),
        conf_matrix_df.loc[:, :].sum().sum(),
      )
      )
print(f'upsides: {upsides.mean()}, top 10 upsides {["%d" % u for u in list(upsides.sort_values(ascending=False).head(10))]}')

desired_up_amount: 42 of 386
up_threshold: 0.4715479153675694
pred len 17943, actuals len 17943, comparison_df len 17943, details len 17943, results_df len 17943
Confusion Matrix:
       up   flat  down
up    11    370     5
flat  34  16969    15
down   0    482    57
ROI1: 173.33% of 100%
ROI2: 97.78% of 100%
Benchmark: 8.61% of 100%
Used: 0.25% (45 of 17943 trades)

upsides: 4.576713417995695, top 10 upsides ['53', '17', '16', '12', '10', '8', '8', '6', '5', '5']


In [694]:
print("Confusion Matrix:\n", conf_matrix_df)
print("ROI1: %.2f%% of 100%%\n"
      "ROI2: %.2f%% of 100%%\n"
      "Benchmark: %.2f%% of 100%%\n"
      "Used: %.2f%% (%d of %d trades)\n"
      %(
        (100 * (PRICE_THRESHOLD * 1. * conf_matrix_df.loc['up', 'up'].sum() + conf_matrix_df.loc['flat', 'up'].sum()) / conf_matrix_df.loc[:, 'up'].sum()),
        (100 * (PRICE_THRESHOLD * 1. * conf_matrix_df.loc['up', 'up'].sum()) / conf_matrix_df.loc[:, 'up'].sum()),
        (100 * (PRICE_THRESHOLD * 1. * conf_matrix_df.loc['up', :].sum()) / conf_matrix_df.loc[:, :].sum().sum()),
        (100 * (1. * conf_matrix_df.loc[:, 'up'].sum()) / conf_matrix_df.loc[:, :].sum().sum()),
        conf_matrix_df.loc[:, 'up'].sum(),
        conf_matrix_df.loc[:, :].sum().sum(),
      )
      )
print(f'upsides: {upsides.mean()}, top 10 upsides {["%d" % u for u in list(upsides.sort_values(ascending=False).head(10))]}')


Confusion Matrix:
       up   flat  down
up    11    370     5
flat  34  16969    15
down   0    482    57
ROI1: 173.33% of 100%
ROI2: 97.78% of 100%
Benchmark: 8.61% of 100%
Used: 0.25% (45 of 17943 trades)

upsides: 4.576713417995695, top 10 upsides ['53', '17', '16', '12', '10', '8', '8', '6', '5', '5']


In [681]:
# results_df
results_df[['mint', 'hour', 'price', 'price'+prediction_type['suffix'], 'upside']]

,mint,hour,price,price_24h,upside
0,4icEZCrEYNop2ZaMMCkRHaNzkt6xG9BpijMCQV7mpw6Z,2023-12-10 22:00:00,3.857099e-05,3.466783e-05,0.898806
1,4k3Dyjzvzp8eMZWUXbBCjEvwSkkk59S5iCNLY3QrkX6R,2023-12-10 22:00:00,1.017082e-02,1.027790e-02,1.010529
2,53ctv3wwFXQbXruKWsbQcCe7sefowyu96pXK6FRLTjfv,2023-12-10 22:00:00,2.935986e-06,4.230865e-06,1.441037
3,5ritAPtFPqQtEFHcHVqNjR5oFNUJqcmgKtZyPd2AyLLy,2023-12-10 22:00:00,3.566448e-07,3.864511e-07,1.083574
4,5yxNbU8DgYJZNi3mPD9rs4XLh9ckXrhPjJ5VCujUWg5H,2023-12-10 22:00:00,1.510666e-09,1.068960e-09,0.707608
...,...,...,...,...,...
17938,gxNuJSHRScR7WkzQEYqvxVditaKkcc9V32KXPJLCbcT,2024-01-01 19:00:00,6.392509e-05,8.026535e-05,1.255616
17939,iotEVVZLEywoTn1QdwNPddxPWszn3zFhEot3MfL9fns,2024-01-01 19:00:00,1.801446e-05,1.709458e-05,0.948937
17940,jtojtomepa8beP8AuQc6eXt5FriJwfFMwQx2v2f9mCL,2024-01-01 19:00:00,1.811443e-02,1.742858e-02,0.962138
17941,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,2024-01-01 19:00:00,1.155330e+00,1.153767e+00,0.998647


In [682]:
# feature importance
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,rsi_1h,11.706403
1,sma_vol_1d,8.554256
2,rsi_4h,8.437217
3,ema_vol_1h,7.939414
4,sma_1d,6.878882
5,sma_vol_1h,6.821743
6,ema_1d,6.427885
7,ema_vol_1d,6.184165
8,ema_vol_4h,6.180397
9,sma_4h,5.508012


In [683]:
guessed = results_df[(results_df['Actual']=='up') & (results_df['Predicted']=='up')].sort_values(by=['mint', 'hour'])
guessed[['mint', 'hour', 'price', 'price'+prediction_type['suffix'], 'upside']]

,mint,hour,price,price_24h,upside
7427,6Xemh445fWbG7C8HpHRbfvTwc2nucuCtPq1WEAAbaRtd,2023-12-21 05:00:00,1.206808e-08,6.424759e-07,53.237626
5334,6fdCC8xfrXNy6PmNaVcxdEY5XNCTAha2V54zYYnmBCey,2023-12-19 09:00:00,9.978044e-07,6.427538e-06,6.441681
15354,6sWet5H2qNpXnV5CwxSmRThx4z3uVtdjHqRB4VyMkJ8Z,2023-12-29 20:00:00,4.054676e-07,1.697392e-06,4.186257
6115,7iT1GRYYhEop2nV1dyCwK2MGyLmPHq47WhPGSwiqcUg5,2023-12-20 02:00:00,6.077024e-08,1.008810e-06,16.600387
6025,9yLRWFwhTi9g9kaXqteTBfneLU2yBL6xYsHjktRVFyrL,2023-12-20 00:00:00,4.774916e-07,3.844946e-06,8.052385
5888,E2ppaAxedkdsHPB4GpB2s6doMG7DWUUXBkH5UwNxvLRR,2023-12-19 21:00:00,2.230599e-07,3.827344e-06,17.158365
798,G3vWvAaXPHCnncnyAbq6yBRXqfRtEV3h7vExzasZeT6g,2023-12-12 10:00:00,3.361695e-07,1.942213e-06,5.777480
7260,GztDUVZbtdWJ6ur9E9STs1SMsHgddUt5q3bGtUboE9sr,2023-12-21 01:00:00,1.864968e-07,1.620347e-06,8.688340
3628,JA5KRHnXv2fuPjQoGwCBw2RXKsbiWMDJ9adPqpZUMDzV,2023-12-17 05:00:00,6.609408e-05,3.829702e-04,5.794319
15550,gxNuJSHRScR7WkzQEYqvxVditaKkcc9V32KXPJLCbcT,2023-12-30 00:00:00,1.449023e-05,1.837584e-04,12.681537


In [695]:
guessed.mint.describe()

count                                               11
unique                                              11
top       6Xemh445fWbG7C8HpHRbfvTwc2nucuCtPq1WEAAbaRtd
freq                                                 1
Name: mint, dtype: object

In [696]:
purchased = results_df[results_df['Predicted']=='up'].sort_values(by=['mint', 'hour'])
purchased[['mint', 'hour', 'price', 'price'+prediction_type['suffix'], 'upside']].sort_values(by=['hour'], ascending=False)

,mint,hour,price,price_24h,upside
15695,gxNuJSHRScR7WkzQEYqvxVditaKkcc9V32KXPJLCbcT,2023-12-30 04:00:00,1.001932e-04,1.769616e-04,1.766203
15550,gxNuJSHRScR7WkzQEYqvxVditaKkcc9V32KXPJLCbcT,2023-12-30 00:00:00,1.449023e-05,1.837584e-04,12.681537
15354,6sWet5H2qNpXnV5CwxSmRThx4z3uVtdjHqRB4VyMkJ8Z,2023-12-29 20:00:00,4.054676e-07,1.697392e-06,4.186257
14634,EVoLsbmQXT6R3b11WjPpGEXyCjw1zmmir271XqDbKRsg,2023-12-29 01:00:00,4.035982e-04,9.493233e-04,2.352149
13483,DdNYDNeqF9MgLEWMUAqQpYesH7uAaK8biNLaiNvCD2TR,2023-12-27 21:00:00,2.648762e-06,5.828702e-06,2.200538
12698,E2WYCGJJtWBodVLy1NKcN8ve4UAtsJJBU2mdErbXxP8h,2023-12-27 03:00:00,7.308997e-08,5.756745e-08,0.787625
12610,CUwif1FiX5b3bwwb2n5Bm35AixvnR8LJjGUVmEwNZNgR,2023-12-27 01:00:00,3.978199e-06,8.698667e-06,2.186584
10848,GHK2LWebz1uFNSd3FrDrwwJwzs6nDxgmce3bJxMYwoQi,2023-12-25 07:00:00,1.189862e-05,9.624835e-06,0.808903
7826,8nTJvVeeQf4n728YWqBF7dX6AoUhqGnxhtuHPQDBEWGJ,2023-12-21 13:00:00,1.895175e-05,1.775148e-05,0.936667
7427,6Xemh445fWbG7C8HpHRbfvTwc2nucuCtPq1WEAAbaRtd,2023-12-21 05:00:00,1.206808e-08,6.424759e-07,53.237626


In [693]:
purchased.mint.describe()


count                                              45
unique                                             36
top       rockaPXZqTqtt9LvznNopikNQn3YSTMwTC4swr7HAiy
freq                                                2
Name: mint, dtype: object

In [360]:
results_df[results_df['Predicted']=='up'].sma_vol_1h.describe()
# fix it


count    488.000000
mean       5.342037
std       10.948481
min        0.007143
25%        0.292289
50%        2.135455
75%        4.358562
max       86.023002
Name: sma_vol_1h, dtype: float64

In [361]:
results_df[(results_df['Actual']=='up') & (results_df['Predicted']=='up')].sma_vol_1h.describe()


count    43.000000
mean      4.515062
std      11.375974
min       0.025854
25%       0.408001
50%       1.589663
75%       3.144892
max      73.194186
Name: sma_vol_1h, dtype: float64

In [362]:
results_df[results_df['Actual']=='up'].sma_vol_1h.describe()

count    1648.000000
mean        6.710630
std        20.849291
min         0.007143
25%         0.178021
50%         0.860083
75%         4.247758
max       333.278670
Name: sma_vol_1h, dtype: float64

In [165]:
results_df['Predicted'].value_counts()

Predicted
flat    24975
down      265
up         30
Name: count, dtype: int64

In [412]:
results_df['mint'].value_counts()

mint
DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7YaB1pPB263    491
EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm    434
J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn    345
mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So     341
bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1     330
                                               ... 
8Gr67j6ktcJ2hzYpbnQGE5CQta2PK1E3GjgvWvZXKbRU      0
GCesTN676jxegJmCQXH6jNaadgtUGsbowKmWJP2F79jt      0
GCkfdzfEgKRV7XTmyjFzif5xE2wTD3fbqu5g1nGy3su8      0
GDHk3Hb9rgv6XCvUyHz7AGRFSEtrv3NHDuiKrP2MN6GM      0
A2z4UBnM3qkiFe4qSPhTkoaUPu8Ehe9UdiJ3aUDVovzD      0
Name: count, Length: 2598, dtype: int64

In [414]:
# mint 99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv
results_df[results_df['mint']=='99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv']

,Actual,Predicted,mint,hour,trades,token_volume,sol_volume,price,class_24h,sma_1h,...,rsi_4h,sma_1d,ema_1d,rsi_1d,sma_vol_1h,ema_vol_1h,sma_vol_4h,ema_vol_4h,sma_vol_1d,ema_vol_1d
18576,up,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-27 21:00:00,2,310973.660216,3.231105,0.000010,up,0.071429,...,100.000000,0.071429,0.133333,100.000000,0.230793,0.430814,0.230793,0.430814,0.230793,0.430814
18647,flat,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-27 22:00:00,4,55121.381882,1.559809,0.000028,flat,0.097656,...,100.000000,0.071429,0.133333,100.000000,0.342208,0.581347,0.342208,0.638788,0.342208,0.638788
18725,flat,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-27 23:00:00,2,41787.227666,0.900513,0.000022,flat,0.199662,...,100.000000,0.071429,0.133333,100.000000,0.406530,0.623902,0.406530,0.758857,0.406530,0.758857
18800,flat,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-28 00:00:00,4,59749.547101,2.223383,0.000037,flat,0.187056,...,100.000000,0.071429,0.133333,100.000000,0.565343,0.837166,0.565343,1.055308,0.565343,1.055308
18876,flat,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-28 01:00:00,3,42250.943520,3.151716,0.000075,flat,0.164741,...,100.000000,0.071429,0.133333,100.000000,0.790466,1.145773,0.790466,1.418095,0.790466,1.475537
18952,flat,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-28 02:00:00,5,59627.402609,5.053669,0.000085,flat,0.216423,...,100.000000,0.071429,0.133333,100.000000,1.151442,1.666826,1.151442,2.064187,1.151442,2.149359
19018,flat,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-28 03:00:00,2,72433.625519,6.210194,0.000086,flat,0.285373,...,100.000000,0.071429,0.133333,100.000000,1.595028,2.272608,1.595028,2.876204,1.595028,2.977385
19078,flat,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-28 04:00:00,3,70318.192675,7.026898,0.000100,flat,0.316268,...,100.000000,0.071429,0.133333,100.000000,2.096949,2.906513,2.096949,3.773597,2.096949,3.914305
19144,flat,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-28 05:00:00,6,105692.212029,6.425166,0.000061,flat,0.591317,...,72.987332,0.071429,0.133333,100.000000,2.555889,3.375667,2.555889,4.524472,2.555889,4.770994
19326,flat,flat,99fp2z9TANArLJR4hcwx8fJSzjz7GQwUn7huGf32nDdv,2023-12-28 08:00:00,2,14313.727769,0.896247,0.000063,flat,0.784225,...,57.246451,0.071429,0.133333,100.000000,2.619907,2.316934,2.619907,4.246639,2.619907,4.890493
